In [98]:
def load_data():
    path_dataset = "datas/train.csv"
    
    x = np.genfromtxt(path_dataset, delimiter=",", skip_header=1, max_rows=50)
    x = x[:, 2:]

    predictions_str = np.genfromtxt(path_dataset, delimiter=",", skip_header=1, usecols=1, dtype=str, max_rows=50)
    y = []
    for pred in predictions_str:
        y.append(-1 if pred=='b' else 1)
    y = np.array(y)
    
    return x, y



In [99]:
import datetime
from helpers import *

x, y = load_data()

y


array([ 1, -1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1,
        1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1, -1,
       -1, -1,  1,  1,  1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1,  1])